In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

tqdm().pandas()



In [ ]:
attempts=pd.read_csv('/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00')

In [ ]:
#Filter for users that have only played at least x games
attempts=attempts.loc[attempts.userGamesPlayed.between(2500,4000)]
#attempts = attempts.loc[attempts.userGamesPlayed > 2500]

In [ ]:
puzzle_data = pd.read_csv('/w/225/1/chess/tactics/tactics_problem.csv')
attempts = pd.merge(attempts,puzzle_data[['tactics_problem_id','rating']],on='tactics_problem_id',how='inner')

In [ ]:
attempts.rename(columns={'rating':'problemRating'},inplace=True)

In [21]:
trimmed_data = attempts[['seconds','is_passed','ratingUser','problemRating']]

In [22]:
ratings = [1400,1700,2000,2300]
rating_rad = 100
rating_ranges = []
for rating in ratings:
    rating_ranges.append((rating-rating_rad,rating+rating_rad))
print(rating_ranges)

[(1300, 1500), (1600, 1800), (1900, 2100), (2200, 2400)]


In [23]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def plot_graph(data,rating):
    figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    colors = ['#1dc23b','#f28d18','#931dc2','#c21d1d']
    for index,data in enumerate(probs):
        color = colors[index]
        plt.plot(data,color=color,label=str(ratings[index]))
        
    plt.ylabel('Probability of success')
    plt.xlabel('Time waited >= x')
    plt.title('Success at puzzles for players around {}'.format(rating))
    plt.legend(loc=2)
    plt.savefig('../plots/puzzle_time_success_{}_players.png'.format(rating))
    plt.clf()

In [25]:
time_range = list(range(0,40))
# if A = # of correct attempts and B = # of problems where seconds >= x time and ratings for user and problem are 
#within a certain range. Want to find: P(A|B) = P(A&B)/P(B)
for player_rating in tqdm(rating_ranges):
    probs = []
    print('Generating plot for players around: ' + str(player_rating[0]+rating_rad))
    for tup in rating_ranges:
        print('Doing ratings for puzzles around: ' + str(tup[0]+rating_rad))
        rating_probs = []
        #Get problems within the correct rating range
        rating_trimmed = trimmed_data.loc[(trimmed_data.problemRating.between(tup[0],tup[1])) & 
                                             (trimmed_data.ratingUser.between(player_rating[0],player_rating[1]))]
        print('Num of attempts: ' + str(len(rating_trimmed)))
        #Filter the problemd from rating_trimmed that were attempted after at least time seconds
        for time in time_range:
            b_condition = rating_trimmed.loc[rating_trimmed.seconds >= time]
            rating_probs.append(len(b_condition.loc[b_condition.is_passed == 1])/len(b_condition))
        probs.append(rating_probs)
    #Making plot
    plot_graph(probs,player_rating[0]+rating_rad)

Generating plot for players around: 1400
Doing ratings for puzzles around: 1400
Num of attempts: 14755
Doing ratings for puzzles around: 1700
Num of attempts: 1874
Doing ratings for puzzles around: 2000
Num of attempts: 3491
Doing ratings for puzzles around: 2300
Num of attempts: 2269
Generating plot for players around: 1700
Doing ratings for puzzles around: 1400
Num of attempts: 281319
Doing ratings for puzzles around: 1700
Num of attempts: 33785
Doing ratings for puzzles around: 2000
Num of attempts: 19415
Doing ratings for puzzles around: 2300
Num of attempts: 14516
Generating plot for players around: 2000
Doing ratings for puzzles around: 1400
Num of attempts: 6960094
Doing ratings for puzzles around: 1700
Num of attempts: 1112851
Doing ratings for puzzles around: 2000
Num of attempts: 164081
Doing ratings for puzzles around: 2300
Num of attempts: 40240
Generating plot for players around: 2300
Doing ratings for puzzles around: 1400
Num of attempts: 1850205
Doing ratings for puzzles

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>